# 🔧 AI Telco Troubleshooting Challenge - Track 3

- **Question**: Can our fine-tuned LLM detect and explain unseen network failures? 

    In other words, for the Track 3: Can we build a specialised edge-cloud LLM to troubleshoot network faults?

- **Target**: Enhance the accuracy of Qwen2.5-1.5B-Instruct when answering telco troubleshooting questions in telelogs data.

- **Objective**: Establish a baseline accuracy for Qwen2.5-1.5B-Instruct on telco troubleshooting questions using telelogs data, and quantify performance gains from domain-specific fine-tuning.

In [1]:
!pip install -q vllm==0.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156

In [2]:
%%writefile baseline.py
"""
TeleLogs Track 3 - Baseline Submission
Author: Kodjo Josué AYITEY (Yehoshua)
Generates submission.csv using Qwen2.5-1.5B-Instruct via vLLM
Supports combined Phase 1 + Phase 2 test set
"""
import csv
import os
import re
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple

import pandas as pd
from vllm import LLM, SamplingParams

# =============================================================================
# CONFIGURATION
# =============================================================================
@dataclass
class Config:
    working_dir: Path = Path("/kaggle/working")
    input_dir: Path = Path("/kaggle/input/the-ai-telco-troubleshooting-challenge-dataset")

    model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"
    tensor_parallel_size: int = 2
    gpu_memory_utilization: float = 0.90
    max_model_len: int = 8192

    temperature: float = 0.7
    top_p: float = 0.9
    max_tokens: int = 256
    num_generations: int = 4   # Must match sample submission

    output_path: Path = working_dir / "submission.csv"
    sample_submission_path: Path = input_dir / "SampleSubmission.csv"

    phase1_test_path: Path = input_dir / "phase_1_test.csv"
    phase2_test_path: Path = input_dir / "phase_2_test.csv"

    target_column: str = "Qwen2.5-1.5B-Instruct"
    placeholder: str = "placeholder"


# =============================================================================
# ROBUST ANSWER EXTRACTION
# =============================================================================
def extract_answer(generated: str, fallback: str = "C8", verbose: bool = False) -> Tuple[str, str]:
    """
    Enhanced extraction to handle varied formats: C1-C8, M1-M5, P1-P8, 1-8, A-I, etc.

    Returns: Tuple[str, str]: (predicted_answer, extraction_method_used)
    """
    text = generated.strip()

    if verbose:
        print(f"\n[DEBUG] Full generated text length: {len(text)} chars")

    # ═══════════════════════════════════════════════════════════
    # Priority 1: \boxed{...} - Most strict, most reliable
    # ═══════════════════════════════════════════════════════════
    boxed_pattern = r'\\boxed\s*(?:\{|\s*)([^}]+?)(?:\s*\}|$)'
    boxed_matches = list(re.finditer(boxed_pattern, text, re.IGNORECASE))

    if boxed_matches:
        for match in reversed(boxed_matches):
            content = match.group(1).strip()
            if verbose:
                print(f"[DEBUG] Found \\boxed{{{content}}}")

            # Flexible match: [A-Z]?[1-9] or [A-I]
            code_match = re.search(r'\b([A-Z]?\d|[A-I])\b', content, re.IGNORECASE)
            if code_match:
                return code_match.group(1).upper(), "boxed"

            # Prefix like "Option B" or "2:"
            code_prefix = re.match(r'^\s*([A-Z]?\d|[A-I])\b', content, re.IGNORECASE)
            if code_prefix:
                return code_prefix.group(1).upper(), "boxed_prefix"

        if verbose:
            print(f"[DEBUG] Found \\boxed{{}} but no valid code inside")

    # ═══════════════════════════════════════════════════════════
    # Priority 2: <answer>...</answer> tags
    # ═══════════════════════════════════════════════════════════
    answer_pattern = r'<answer>\s*([^<]+?)\s*</answer>'
    answer_blocks = re.findall(answer_pattern, text, re.IGNORECASE | re.DOTALL)

    if answer_blocks:
        for block in reversed(answer_blocks):
            if verbose:
                print(f"[DEBUG] Found <answer>{block.strip()}</answer>")

            code_match = re.search(r'\b([A-Z]?\d|[A-I])\b', block, re.IGNORECASE)
            if code_match:
                return code_match.group(1).upper(), "answer_tag"

    # ═══════════════════════════════════════════════════════════
    # Priority 3: Final answer keywords followed by code
    # ═══════════════════════════════════════════════════════════
    final_keywords = [
        r'final\s+answer\s*:?\s*([A-Z]?\d|[A-I])\b',
        r'answer\s*:?\s*([A-Z]?\d|[A-I])\b',
        r'therefore\s*,?\s*(?:the\s+)?(?:answer\s+is\s*)?([A-Z]?\d|[A-I])\b',
        r'conclusion\s*:?\s*([A-Z]?\d|[A-I])\b',
        r'select\s*:?\s*([A-Z]?\d|[A-I])\b',
        r'choose\s*:?\s*([A-Z]?\d|[A-I])\b',
    ]

    for pattern in final_keywords:
        matches = list(re.finditer(pattern, text, re.IGNORECASE))
        if matches:
            last = matches[-1].group(1).upper()
            if verbose:
                print(f"[DEBUG] Keyword match: {last}")
            return last, "keyword"

    # ═══════════════════════════════════════════════════════════
    # Priority 4: Last sentence containing "<code> is" or "<code> because"
    # ═══════════════════════════════════════════════════════════
    sentences = re.split(r'[.!?]\s+', text)
    for sentence in reversed(sentences):
        if re.search(r'\b([A-Z]?\d|[A-I])\s+(?:is|because|explains|accounts)\b', sentence, re.IGNORECASE):
            code_match = re.search(r'\b([A-Z]?\d|[A-I])\b', sentence, re.IGNORECASE)
            if code_match:
                return code_match.group(1).upper(), "contextual_sentence"

    # ═══════════════════════════════════════════════════════════
    # Priority 5: Last standalone code in text (weakest fallback)
    # ═══════════════════════════════════════════════════════════
    code_matches = re.findall(r'\b([A-Z]?\d|[A-I])\b', text, re.IGNORECASE)

    if code_matches:
        last_code = code_matches[-1].upper()
        unique_codes = set(c.upper() for c in code_matches)

        if len(unique_codes) > 1 and verbose:
            print(f"[DEBUG] Multiple codes: {unique_codes}, using last: {last_code}")

        return last_code, "last_occurrence"

    # ═══════════════════════════════════════════════════════════
    # Ultimate fallback
    # ═══════════════════════════════════════════════════════════
    if verbose:
        print("[WARNING] No valid code found → using fallback")

    return fallback, "fallback"


def format_answer(cause: str) -> str:
    return f"Based on the provided data, the most likely root cause for the issue is: \\boxed{{{cause}}}"


# =============================================================================
# MAIN PIPELINE
# =============================================================================
def main():
    config = Config()
    config.working_dir.mkdir(exist_ok=True)

    print("🚀 Starting baseline submission generation...")

    # Load sample submission template
    if not config.sample_submission_path.exists():
        raise FileNotFoundError(f"SampleSubmission.csv not found at {config.sample_submission_path}")

    submission_df = pd.read_csv(config.sample_submission_path)
    print(f"Loaded submission template with {len(submission_df)} rows")

    # Load test questions from both phases
    test_files = []
    if config.phase1_test_path.exists():
        test_files.append(config.phase1_test_path)
    if config.phase2_test_path.exists():
        test_files.append(config.phase2_test_path)

    if not test_files:
        raise FileNotFoundError("Neither phase_1_test.csv nor phase_2_test.csv found.")

    dfs = [pd.read_csv(p) for p in test_files]
    test_df = pd.concat(dfs, ignore_index=True)
    test_df = test_df.drop_duplicates(subset=["ID"])  # safety
    print(f"Loaded {len(test_df)} base questions from test set(s)")

    # Prepare prompts
    prompts = [f"<|im_start|>user\n{row['question']}<|im_end|>\n<|im_start|>assistant\n"
               for _, row in test_df.iterrows()]
    base_ids = test_df["ID"].tolist()

    # Initialize model
    print(f"Loading model {config.model_name}...")
    llm = LLM(
        model=config.model_name,
        tensor_parallel_size=config.tensor_parallel_size,
        gpu_memory_utilization=config.gpu_memory_utilization,
        max_model_len=config.max_model_len,
        trust_remote_code=True,
        enforce_eager=True,
        dtype="float16",
        disable_custom_all_reduce=True,
    )

    sampling_params = SamplingParams(
        temperature=config.temperature,
        top_p=config.top_p,
        max_tokens=config.max_tokens,
        n=config.num_generations,
    )

    # Generate responses
    print("Generating responses...")
    outputs = llm.generate(prompts, sampling_params, use_tqdm=True)

    # Fill submission
    filled_count = 0
    for base_id, output in zip(base_ids, outputs):
        for i, completion in enumerate(output.outputs, 1):
            row_id = f"{base_id}_{i}"
            cause, method = extract_answer(completion.text, fallback="placeholder")
            # Optional: print(f"{row_id} → {cause} ({method})")

            answer_text = format_answer(cause)

            mask = submission_df["ID"] == row_id
            if mask.any():
                submission_df.loc[mask, config.target_column] = answer_text
                filled_count += 1

    # Fill placeholder columns
    for col in submission_df.columns:
        if col not in ["ID", config.target_column]:
            submission_df[col] = config.placeholder

    # Save
    submission_df.to_csv(config.output_path, index=False, quoting=csv.QUOTE_ALL)
    print(f"✅ Submission saved to {config.output_path}")
    print(f"   Filled {filled_count} rows for {config.target_column}")
    print(f"   Total rows in submission: {len(submission_df)}")


if __name__ == "__main__":
    main()

Writing baseline.py


In [3]:
!python baseline.py

2026-01-25 15:10:47.369103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769353847.573907      82 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769353847.635565      82 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769353848.115044      82 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769353848.115085      82 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769353848.115092      82 computation_placer.cc:177] computation placer alr

In [4]:
# 